In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import POLOCM2BASELINE
from utils import read_plan
from evaluator import ExecutabilityEvaluator

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

In [ ]:
def test_exe(domain_name, trace, test_trace=None):
    debug = {
        # 'get_transition_sets': True,
        # 'extract_holes': True,
        # 'find_holes': True,
    }
    try:
        locm2 = POLOCM2BASELINE(state_param=True, viz=True, debug=debug)
        model = locm2.extract_model([trace])
        
        actions = model.actions
        
        pddl_domain = model.to_pddl_domain("test_bw")
        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            exe = evaluator.get_overall_executability('l', test_trace, set(), set())
        else:
            exe = evaluator.get_overall_executability('l', trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [3]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)

# test_exe("plan1", trace)
test_exe("plan2", po_trace)
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

action <PropositionalAction '(pickup b2)' at 0x2fef46526f0> not executable
preconditions not satisfied:  {<Atom s1f2_state1(b2)>}
Warning! domain: plan2 - executability: 0.75


0.75

In [4]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         details = line.split("&&")
#         name = f"{details[0]}-{details[2]}-{details[3]}"
      
#         plan = details[-1]
#         trace = read_plan(plan)
#         po_trace = trace.to_partial_ordered_trace(0.3)
#         test_exe(name, po_trace)